In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

#load data

#Lineares Model fitting
# Fit the linear model
mod = smf.ols('price ~ sqft_lot15 + C(condition)', data=house_pricing).fit()

# Print the summary
print(mod.summary())
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, t

# Wochentag aus Datum berechnen
df_umsatz_kiwo_wetter['Wochentag'] = pd.to_datetime(df_umsatz_kiwo_wetter['Datum']).dt.day_name()

# Definieren der Warengruppen-Beschriftungen
Warengruppe = [1, 2, 3, 4, 5, 6]
Backwaren = ["Brot", "Brötchen", "Croissant", "Konditorei", "Kuchen", "Saisonbrot"]

# Konfidenzintervall-Berechnung für jede Warengruppe
confidence = 0.95  # 95% Konfidenzintervall

# Detaillierte Konfidenzintervall-Berechnung
def calculate_confidence_interval(group):
    mean = group['Umsatz'].mean()
    count = len(group)
    std = group['Umsatz'].std()
    standard_error = std / np.sqrt(count)
    t_value = t.ppf((1 + confidence) / 2, count - 1)
    ci_width = t_value * standard_error
    return pd.Series({
        'mean': mean,
        'ci_lower': mean - ci_width,
        'ci_upper': mean + ci_width
    })

# Gruppierung nach Wochentag und Warengruppe mit Konfidenzintervallen
summary = df_umsatz_kiwo_wetter.groupby(['Wochentag', 'Warengruppe']).apply(calculate_confidence_interval).reset_index()

# Ersetzen der Warengruppen-Nummern durch Backwaren-Namen
summary['Warengruppe'] = summary['Warengruppe'].map(dict(zip(Warengruppe, Backwaren)))

# Sortierung der Wochentage
wochentage_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Visualisierung mit Konfidenzintervallen für jede Warengruppe
# Erhöhter vertikaler Platz für den Titel
plt.figure(figsize=(15, 9), constrained_layout=True)

# Liste der Warengruppen
warengruppen = summary['Warengruppe'].unique()

# Subplots für jede Warengruppe
for i, gruppe in enumerate(warengruppen, 1):
    plt.subplot(2, 3, i)
    gruppe_data = summary[summary['Warengruppe'] == gruppe]
   
    # Sortieren nach Wochentagsreihenfolge
    gruppe_data = gruppe_data.set_index('Wochentag').loc[wochentage_order].reset_index()
   
    plt.bar(gruppe_data['Wochentag'], gruppe_data['mean'], yerr=(gruppe_data['mean'] - gruppe_data['ci_lower'], gruppe_data['ci_upper'] - gruppe_data['mean']),
            capsize=5, color='skyblue', edgecolor='navy')
   
    plt.title(f'Umsatz {gruppe}')
    plt.xlabel('Wochentag')
    plt.ylabel('Durchschnittlicher Umsatz (€)')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

# Titel mit mehr Abstand
plt.suptitle('Umsatz pro Warengruppe mit 95% Konfidenzintervallen', fontsize=16, y=1.02)
plt.show()